In [1]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import load_model
import joblib
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


In [2]:
loaded_model = load_model('trained_model.h5')
loaded_target_scaler = joblib.load('scaler.pkl')
df = pd.read_csv('./rawData/data_prueba_Forecasting_validation_data.csv')
df=df.drop(['Unnamed: 0'], axis=1)

new_data_features = df[['Evento', 'Tienda', 'Producto', 'Mes', 'Day', 'Year', 'Formato_Hiper', 'Formato_Super',
                                 'Subformato_Basico', 'Subformato_Intermedio', 'Subformato_Plus', 'DayOfWeek']]


In [3]:
new_data_features

,Evento,Tienda,Producto,Mes,Day,Year,Formato_Hiper,Formato_Super,Subformato_Basico,Subformato_Intermedio,Subformato_Plus,DayOfWeek
0,75,1,369772,10,15,2021,0,1,1,0,0,4
1,76,1,369772,10,16,2021,0,1,1,0,0,5
2,77,1,369772,10,17,2021,0,1,1,0,0,6
3,78,1,369772,10,18,2021,0,1,1,0,0,0
4,79,1,369772,10,19,2021,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
78764,87,1,2098857,10,27,2021,0,1,0,0,1,2
78765,88,1,2098857,10,28,2021,0,1,0,0,1,3
78766,89,1,2098857,10,29,2021,0,1,0,0,1,4
78767,90,1,2098857,10,30,2021,0,1,0,0,1,5


In [4]:
loaded_model.summary()


Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_27 (LSTM)              (None, 50)                12600     
                                                                 
 dense_27 (Dense)            (None, 1)                 51        
                                                                 
Total params: 12,651
Trainable params: 12,651
Non-trainable params: 0
_________________________________________________________________


In [5]:
df.head()

,Venta,Evento,Tienda,Producto,id_fec_diaria,Precio,Mes,Day,Year,Formato_Hiper,Formato_Super,Subformato_Basico,Subformato_Intermedio,Subformato_Plus,DayOfWeek
0,1.5,75,1,369772,2021-10-15,26.030001,10,15,2021,0,1,1,0,0,4
1,1.0,76,1,369772,2021-10-16,26.030001,10,16,2021,0,1,1,0,0,5
2,4.0,77,1,369772,2021-10-17,26.030001,10,17,2021,0,1,1,0,0,6
3,3.0,78,1,369772,2021-10-18,25.636667,10,18,2021,0,1,1,0,0,0
4,2.5,79,1,369772,2021-10-19,25.693334,10,19,2021,0,1,1,0,0,1


In [6]:
df.shape

(78769, 15)